In [3]:
import pandas as pd
import numpy as np
from Bio import SeqIO # pip install BioPython
from Bio import pairwise2 # Untuk alignment
from Bio.pairwise2 import format_alignment # Untuk format tampilan alignment
import re
import os
import gdown # pip install gdown

# =============================================================================
# KONSTANTA DAN KONFIGURASI
# =============================================================================
FOCUSED_GENES = [
    'ATM', 'BARD1', 'BRCA1', 'BRCA2', 'BRIP1', 'CDH1', 'CHEK2', 'EPCAM',
    'MLH1', 'MRE11',
    'MSH2', 'MSH6', 'NBN', 'PALB2', 'PMS2', 'PTEN',
    'RAD50', 'RAD51C', 'RAD51D', 'XRCC2'
]

# =============================================================================
# ID GOOGLE DRIVE (Pastikan ID ini benar dan file dapat diakses publik)
# =============================================================================
FASTA_FILE_GDRIVE_ID = "1nxQKyLEiW0UYrBHHbJ4uQrLhao25wNqq"
GENE_TSV_GDRIVE_IDS = {
    'ATM': '1_Spv8rRT3OzY2UR6g-pgEmzqpiV8k032', 'BARD1': '10b_1D59t-_cNti9dmOFbKQLZuLGS18eV',
    'BRCA1': '1AfRjytUsijtXdDyM1JNg1PUI3HxurIPe', 'BRCA2': '1VJjF3cy7idadtLnVNzg0BPtnHR8Zolqu',
    'BRIP1': '1XY2buF2MGFA2uOO2Nk05e3fGEeVkCIRr', 'CDH1': '1aIa5LoNOpfJUQAimepT8Xwho6euuqFue',
    'CHEK2': '1VHJkKCILDGScvDHEuxm44lJegbHRgGql', 'EPCAM': '1puRuxXC5-vcuedoT66Wp9xm0gWL34lS_',
    'MLH1': '10nxKqYpF2wzl6nAVmZShhrqccBCGQplO', 'MRE11': '16zQoIkyEEXrO6L7z1bEaVENw4PeMWXuW',
    'MSH2': '1ffz3aMyfAKcBAdUIw0W-rkqYAjgPEmng', 'MSH6': '1-MrETYFJdNMcmZrqTlmztsYx4xrPhIla',
    'NBN': '1TV4wBg178H4Z_v8IGJNPNepPsgqDz197', 'PALB2': '1i89bwbZdq5ZahgzeC8DnoEY4RKISjhW3',
    'PMS2': '1jrKVVt4nEfD9Fiwvux876aV-sVPbJPHN', 'PTEN': '1HYzWCDS6up8rj5_ZD55Q5bM53Y-3gYrd',
    'RAD50': '14QnJ14F-PnweSbTBS3cIRge9wBfwY2MQ', 'RAD51C': '1T-9l7Szpk23vOG6qYSJHmCjUHbo_bREn',
    'RAD51D': '1Wokxxcdliwt-_wLCQ6h54bx4s2UUENJ_', 'XRCC2': '1dyJUJau6Xk_uUN7TNR0tup6aMcjzicFE'
}

# =============================================================================
# FUNGSI-FUNGSI PEMBANTU
# =============================================================================
def parse_fasta_header(header):
    try:
        parts = header.split()
        transcript_id = parts[1]
        chrom_info = parts[2]
        chrom = chrom_info.split(":")[0]
        # Mencoba menangkap format seperti 1:65419-71585(+) atau GRCh38:CM000663.2:1:65419-71585(+)
        match = re.search(r'(\d+)-(\d+)\((.)\)', chrom_info)
        if match:
            start, end = int(match.group(1)), int(match.group(2))
            strand = match.group(3)
            return transcript_id, chrom, start, end, strand
        else: # Fallback untuk format header yang mungkin hanya :start-end
            match_alt = re.search(r':(\d+)-(\d+)', chrom_info)
            if match_alt:
                start, end = int(match_alt.group(1)), int(match_alt.group(2))
                strand_match = re.search(r'\((.)\)', chrom_info) # Mencoba mencari strand jika ada
                strand = strand_match.group(1) if strand_match else '+' # Asumsi + jika tidak ada
                return transcript_id, chrom, start, end, strand
        # print(f"Warning: Could not fully parse header coordinates/strand: {header}")
        return transcript_id, chrom, None, None, None # Kembalikan ID dan kromosom jika sebagian terparsir
    except (IndexError, AttributeError, ValueError):
        # print(f"Warning: Exception parsing header: {header}")
        return None, None, None, None, None

def load_fasta_sequences_with_pos(fasta_path):
    seq_dict = {}
    pos_dict = {}
    if not os.path.exists(fasta_path):
        print(f"FASTA file not found at {fasta_path}.")
        return seq_dict, pos_dict
    try:
        for record in SeqIO.parse(fasta_path, "fasta"):
            transcript_id, chrom, start, end, strand = parse_fasta_header(record.description)
            if transcript_id and start is not None and end is not None and strand is not None: # Pastikan semua info posisi ada
                seq_dict[transcript_id] = str(record.seq).upper()
                pos_dict[transcript_id] = (start, end, strand)
            elif transcript_id:
                print(f"Warning: Missing full position info for transcript {transcript_id} in FASTA, skipping.")
    except Exception as e:
        print(f"Error parsing FASTA file {fasta_path}: {e}")
    return seq_dict, pos_dict

def load_all_mutation_data(data_directory, gene_list, gene_gdrive_ids_map):
    all_mutations_list = []
    # print(f"Attempting to load/download mutation data for: {gene_list}")
    for gene_name in gene_list:
        file_path = os.path.join(data_directory, f"{gene_name}.tsv")
        gdrive_id = gene_gdrive_ids_map.get(gene_name)
        needs_download = False
        if not os.path.exists(file_path):
            if gdrive_id and gdrive_id != f"ID_GOOGLE_DRIVE_ANDA_UNTUK_{gene_name}.tsv": # Pastikan ID valid
                needs_download = True
            else:
                print(f"Warning: Mutation file {file_path} not found locally and no valid GDrive ID for {gene_name}.")
                continue
        if needs_download:
            print(f"Downloading {file_path} for gene {gene_name} from GDrive ID: {gdrive_id}...")
            try:
                if os.path.exists(file_path): os.remove(file_path)
                gdown.download(f"https://drive.google.com/uc?id={gdrive_id}", file_path, quiet=False)
                if not os.path.exists(file_path):
                     print(f"Download seems to have failed for {file_path}.")
                     continue
            except Exception as e:
                print(f"Failed to download {file_path}: {e}")
                continue
        if os.path.exists(file_path):
            try:
                df_gene = pd.read_csv(file_path, sep='\t', low_memory=False)
                all_mutations_list.append(df_gene)
            except Exception as e:
                print(f"Error reading mutation file {file_path}: {e}")

    if not all_mutations_list:
        print("No mutation files were successfully loaded.")
        return pd.DataFrame()
    df_combined = pd.concat(all_mutations_list, ignore_index=True)
    print(f"Total records from all TSV files: {len(df_combined)}")
    return df_combined

def get_cds_mutation_details(cds_mutation_str):
    if not cds_mutation_str or pd.isna(cds_mutation_str) or cds_mutation_str == 'c.?':
        return None

    cds_mutation_str = str(cds_mutation_str)

    # SNV
    snv_match = re.search(r'c\.((?:_?\*?-?\d+(?:[+-]\d+)?)?)?([ACGTN])>([ACGTN])', cds_mutation_str, re.IGNORECASE)
    if snv_match:
        pos_full_str = snv_match.group(1) if snv_match.group(1) else ""
        main_pos_cds = 0
        numeric_part_of_pos = "0"
        if pos_full_str:
            main_num_match = re.search(r'(\d+)', pos_full_str)
            if main_num_match: numeric_part_of_pos = main_num_match.group(1)
        try:
            main_pos_cds = int(numeric_part_of_pos)
        except ValueError:
            if pos_full_str and numeric_part_of_pos == "0" and not re.search(r'\b0\b', pos_full_str) : pass
            elif numeric_part_of_pos != "0": print(f"Warning (SNV): Could not convert '{numeric_part_of_pos}' from '{pos_full_str}'. CDS: {cds_mutation_str}")
        return {'type': 'SNV', 'pos_cds_str': pos_full_str,
                'pos_cds_numeric': main_pos_cds, 'ref': snv_match.group(2), 'alt': snv_match.group(3)}

    # Deletion
    del_match = re.search(r'c\.(\*?-?\d+(?:[+-]\d+)?)(?:_(\*?-?\d+(?:[+-]\d+)?))?del([ACGTN]*)?', cds_mutation_str, re.IGNORECASE)
    if del_match:
        start_pos_str = del_match.group(1); end_pos_str = del_match.group(2) if del_match.group(2) else start_pos_str
        deleted_bases = del_match.group(3) if del_match.group(3) else '' # Grup 3 untuk del([ACGTN]*)
        main_start_pos_cds = 0; main_num_match_start = re.search(r'(\d+)', start_pos_str);
        if main_num_match_start: main_start_pos_cds = int(main_num_match_start.group(1))
        main_end_pos_cds = main_start_pos_cds;
        if end_pos_str != start_pos_str :
            main_num_match_end = re.search(r'(\d+)', end_pos_str);
            if main_num_match_end: main_end_pos_cds = int(main_num_match_end.group(1))
        len_del_val = (main_end_pos_cds - main_start_pos_cds + 1) if main_end_pos_cds >= main_start_pos_cds else 0
        if not deleted_bases and len_del_val == 0: len_del_val = 1
        elif deleted_bases: len_del_val = len(deleted_bases)
        return {'type': 'DELETION', 'pos_cds_str': f"{start_pos_str}_{end_pos_str}" if start_pos_str != end_pos_str else start_pos_str,
                'pos_cds_numeric_start': main_start_pos_cds, 'pos_cds_numeric_end': main_end_pos_cds,
                'deleted_bases': deleted_bases, 'len_change': -len_del_val}

    # Insertion - Complex (pos1_pos2insBASES)
    ins_match_complex = re.search(r'c\.(\*?-?\d+(?:[+-]\d+)?)_(\*?-?\d+(?:[+-]\d+)?)ins([ACGTN]+)', cds_mutation_str, re.IGNORECASE)
    if ins_match_complex:
        pos1_str = ins_match_complex.group(1); pos2_str = ins_match_complex.group(2) # Grup 2 adalah pos2
        inserted_bases = ins_match_complex.group(3) # Grup 3 adalah basa
        main_pos1_cds = 0; main_num_match1 = re.search(r'(\d+)', pos1_str); 
        if main_num_match1: main_pos1_cds = int(main_num_match1.group(1))
        main_pos2_cds = 0; main_num_match2 = re.search(r'(\d+)', pos2_str);
        if main_num_match2: main_pos2_cds = int(main_num_match2.group(1))
        return {'type': 'INSERTION', 'pos_cds_str': f"{pos1_str}_{pos2_str}",
                'pos_cds_numeric_start': main_pos1_cds, 'pos_cds_numeric_end': main_pos2_cds,
                'inserted_bases': inserted_bases, 'len_change': len(inserted_bases)}

    # Insertion - Simple (pos_insBASES)
    ins_match_simple = re.search(r'c\.(\*?-?\d+(?:[+-]\d+)?)ins([ACGTN]+)', cds_mutation_str, re.IGNORECASE)
    if ins_match_simple:
        pos1_str = ins_match_simple.group(1); inserted_bases = ins_match_simple.group(2) # Grup 2 adalah basa
        main_pos1_cds = 0; main_num_match1 = re.search(r'(\d+)', pos1_str);
        if main_num_match1: main_pos1_cds = int(main_num_match1.group(1))
        return {'type': 'INSERTION', 'pos_cds_str': f"{pos1_str}",
                'pos_cds_numeric_start': main_pos1_cds, 'pos_cds_numeric_end': main_pos1_cds,
                'inserted_bases': inserted_bases, 'len_change': len(inserted_bases)}
                
    # Duplication: c.77dup, c.123_125dup, c.*5dup
    # Regex diubah agar grup basa duplikasi (jika ada) menjadi opsional dan tidak error jika tidak ada.
    # Grup: 1=(pos_awal_full), 2=(offset_awal), 3=(?:_pos_akhir_full), 4=(pos_akhir_full tanpa _), 5=(offset_akhir), 6=([ACGTN]*)
    dup_match = re.search(r'c\.(\*?-?\d+(?:[+-]\d+)?)(?:_(\*?-?\d+(?:[+-]\d+)?))?dup([ACGTN]*)?', cds_mutation_str, re.IGNORECASE)
    if dup_match:
        start_pos_str = dup_match.group(1)
        end_pos_str = dup_match.group(2) if dup_match.group(2) else start_pos_str # Jika grup 2 (posisi akhir) tidak ada, berarti duplikasi 1 posisi
        
        # Grup untuk basa yang diduplikasi sekarang adalah grup ke-3 dari regex baru ini
        duplicated_bases_notation = dup_match.group(3) if dup_match.group(3) else ''
        
        main_start_pos_cds = 0; main_num_match_start = re.search(r'(\d+)', start_pos_str);
        if main_num_match_start: main_start_pos_cds = int(main_num_match_start.group(1))
            
        main_end_pos_cds = main_start_pos_cds; # Default jika duplikasi satu posisi
        if end_pos_str != start_pos_str: # Hanya jika ada rentang posisi kedua
            main_num_match_end = re.search(r'(\d+)', end_pos_str);
            if main_num_match_end: main_end_pos_cds = int(main_num_match_end.group(1))

        len_dup_val = (main_end_pos_cds - main_start_pos_cds + 1) if main_end_pos_cds >= main_start_pos_cds else 0
        if not duplicated_bases_notation and len_dup_val == 0: # Misal c.77dup (tidak ada basa disebut, panjangnya 1)
            len_dup_val = 1
        elif duplicated_bases_notation: # Misal c.77dupA (panjangnya 1, basanya A)
            len_dup_val = len(duplicated_bases_notation)
        # Jika c.77_79dup (tidak ada basa disebut, panjangnya 3)
        # Jika c.77_79dupAGT (ada basa disebut, panjangnya 3)

        return {'type': 'DUPLICATION', 
                'pos_cds_str': f"{start_pos_str}_{end_pos_str}" if start_pos_str != end_pos_str else start_pos_str,
                'pos_cds_numeric_start': main_start_pos_cds, 
                'pos_cds_numeric_end': main_end_pos_cds,
                'duplicated_bases': duplicated_bases_notation, 
                'len_change': len_dup_val}
    return None


def reconstruct_patient_sequence_with_mutations(reference_seq_str, mutations_for_transcript,
                                                transcript_pos_dict_entry, cds_col_name):
    patient_seq_list = list(reference_seq_str)
    transcript_start_genome, transcript_end_genome, transcript_strand = transcript_pos_dict_entry
    ref_len_original = len(reference_seq_str)
    offset = 0 
    applied_mutations_info = []

    # Urutkan mutasi berdasarkan GENOME_START, dan untuk strand '-', urutkan descending agar apply dari 'belakang' dulu
    # Ini penting agar offset konsisten.
    mutations_sorted = mutations_for_transcript.sort_values(by='GENOME_START', ascending=(transcript_strand == '+'))

    for _, mut_row in mutations_sorted.iterrows():
        cds_mutation_str = str(mut_row.get(cds_col_name, ''))
        # Pastikan GENOME_START dan GENOME_STOP adalah numerik
        try:
            genome_start_mut = int(mut_row['GENOME_START'])
            genome_end_mut = int(mut_row.get('GENOME_STOP', genome_start_mut))
            if pd.isna(genome_end_mut): genome_end_mut = genome_start_mut # Handle NaN
            if genome_end_mut < genome_start_mut: genome_end_mut = genome_start_mut
        except ValueError:
            # print(f"  Skipping mutation due to invalid GENOME_START/STOP for CDS: {cds_mutation_str}")
            continue
            
        details = get_cds_mutation_details(cds_mutation_str)
        if not details:
            continue

        # Pemetaan posisi genomik mutasi ke indeks sekuens transkrip (0-based) relatif ke AWAL transkrip
        idx_mut_start_in_ref = genome_start_mut - transcript_start_genome
        idx_mut_end_in_ref   = genome_end_mut   - transcript_start_genome
        
        if transcript_strand == '-':
            # Untuk reverse strand, hitung dari akhir transkrip dan balik urutannya
            ref_transcript_len = transcript_end_genome - transcript_start_genome + 1
            
            # Posisi awal mutasi (dari GENOME_START) relatif ke akhir transkrip
            # Posisi akhir mutasi (dari GENOME_STOP) relatif ke akhir transkrip
            # Contoh: ref 100-200. mutasi di 150. Di reverse: (200-150) = pos 50 dari akhir.
            # Indeks list 0-based: (ref_len -1) - (pos_dari_akhir)
            
            original_end_rel_to_start = genome_end_mut - transcript_start_genome
            original_start_rel_to_start = genome_start_mut - transcript_start_genome

            idx_mut_start_in_ref = (ref_transcript_len - 1) - original_end_rel_to_start
            idx_mut_end_in_ref = (ref_transcript_len - 1) - original_start_rel_to_start
            
        # Sesuaikan dengan offset dari indel sebelumnya yang telah diterapkan pada patient_seq_list
        current_idx_start_on_list = idx_mut_start_in_ref + offset
        current_idx_end_on_list   = idx_mut_end_in_ref   + offset


        if details['type'] == 'SNV':
            if 0 <= current_idx_start_on_list < len(patient_seq_list):
                original_base_in_list = patient_seq_list[current_idx_start_on_list]
                patient_seq_list[current_idx_start_on_list] = details['alt']
                applied_mutations_info.append(
                    f"SNV @CDS:'{details['pos_cds_str']}' (Genomic:{genome_start_mut}, ListIdx:{current_idx_start_on_list}): Ref_in_list '{original_base_in_list}'(CDS_ref:'{details['ref']}') -> Alt '{details['alt']}'"
                )
        
        elif details['type'] == 'DELETION':
            len_del = details.get('len_change', 0) * -1
            if 0 <= current_idx_start_on_list < len(patient_seq_list) and current_idx_start_on_list + len_del <= len(patient_seq_list):
                deleted_part = "".join(patient_seq_list[current_idx_start_on_list : current_idx_start_on_list + len_del])
                del patient_seq_list[current_idx_start_on_list : current_idx_start_on_list + len_del]
                offset -= len_del
                applied_mutations_info.append(
                    f"DELETION @CDS:'{details['pos_cds_str']}' (Genomic:{genome_start_mut}-{genome_end_mut}, ListIdxStart:{current_idx_start_on_list}), Len:{len_del}, Bases_in_list:'{deleted_part}'"
                )

        elif details['type'] == 'INSERTION' or details['type'] == 'DUPLICATION':
            bases_to_insert = details.get('inserted_bases', '')
            if details['type'] == 'DUPLICATION' and not details.get('duplicated_bases'):
                # Jika 'duplicated_bases' tidak ada di notasi, coba ambil dari reference_seq_str asli
                # Ini memerlukan pemetaan posisi CDS yang akurat ke ref_seq_str
                # Untuk duplikasi c.X_Ydup, posisi dihitung dari awal CDS.
                # Perlu mapping yang sangat hati-hati dari CDS_pos ke ref_seq_str index.
                # Untuk saat ini, jika duplicated_bases kosong, kita pakai placeholder.
                # Ini adalah area yang paling sulit diimplementasikan dengan akurat tanpa library khusus.
                dup_cds_start_0based = details['pos_cds_numeric_start'] - 1
                dup_cds_end_0based = details['pos_cds_numeric_end'] # Ini sudah eksklusif jika dari A_Bdup
                
                # Jika hanya satu basa c.Xdup -> range [X-1:X]
                if details['pos_cds_numeric_start'] == details['pos_cds_numeric_end'] :
                    dup_cds_end_0based = details['pos_cds_numeric_start']

                # Logika ini masih sangat kasar untuk duplikasi dari referensi,
                # karena posisi CDS tidak langsung memetakan ke genomik jika ada intron.
                # Kita akan gunakan GENOME_START dan GENOME_STOP mutasi untuk mengambil dari referensi.
                if 0 <= idx_mut_start_in_ref < idx_mut_end_in_ref +1 <= ref_len_original :
                     bases_to_insert = reference_seq_str[idx_mut_start_in_ref : idx_mut_end_in_ref +1]
                else:
                     bases_to_insert = "DUP_BASES_UNKNOWN" # Placeholder

            if not bases_to_insert:
                continue

            # Insersi umumnya setelah posisi CDS. Jika c.N_N+1ins, insersi setelah pos N+1 di CDS.
            # Jika c.Nins, insersi setelah N.
            # current_idx_start_on_list adalah indeks *awal* dari rentang yang terpengaruh pada sekuens yang dimodifikasi.
            # Untuk insersi, kita taruh *setelah* posisi tersebut.
            insert_at_idx_on_list = current_idx_start_on_list + 1 # Default: setelah posisi pertama yang terlibat
            if details['type'] == 'DUPLICATION':
                insert_at_idx_on_list = current_idx_end_on_list + 1 # Duplikasi terjadi setelah rentang


            if 0 <= insert_at_idx_on_list <= len(patient_seq_list):
                patient_seq_list = patient_seq_list[:insert_at_idx_on_list] + list(bases_to_insert) + patient_seq_list[insert_at_idx_on_list:]
                offset += len(bases_to_insert)
                applied_mutations_info.append(
                    f"{details['type']} @CDS:'{details['pos_cds_str']}' (Genomic:{genome_start_mut}, ListIdxAfter:{insert_at_idx_on_list-1}), Bases:'{bases_to_insert}'"
                )
                
    return "".join(patient_seq_list), applied_mutations_info


def compare_and_display_sequences(patient_seq, reference_seq, transcript_id, mutations_info):
    print(f"\n--- Comparison for Transcript: {transcript_id} ---")
    print(f"Reference Length: {len(reference_seq)}, Patient Length: {len(patient_seq)}")
    if mutations_info:
        print("Applied Mutations (attempted reconstruction):")
        for info in mutations_info:
            print(f"  - {info}")
    elif patient_seq == reference_seq:
         print("  No recognized mutations applied, and sequences are identical.")
    else:
         print("  No recognized mutations applied, but sequences differ (reconstruction might be incomplete or other variations exist).")

    print("\nSequence Alignment (Global - Needleman-Wunsch like):")
    if not reference_seq and not patient_seq:
        print("  Both sequences are empty. Cannot align.")
        return
    if not reference_seq:
        print(f"  Reference sequence is empty. Patient (first 100bp): {patient_seq[:100]}")
        return
    if not patient_seq:
        print(f"  Patient sequence is empty. Reference (first 100bp): {reference_seq[:100]}")
        return

    # Gunakan alignment dari BioPython untuk hasil yang lebih baik
    try:
        # Parameter alignment: match=2, mismatch_penalty=-1, gap_open_penalty=-2, gap_extend_penalty=-0.5
        # Ini adalah parameter umum, bisa disesuaikan.
        alignments = pairwise2.align.globalms(reference_seq, patient_seq, 2, -1, -2, -0.5)
    except Exception as e:
        print(f"  Could not perform pairwise alignment for {transcript_id}: {e}")
        print("  Displaying raw sequences (first 100bp) for manual comparison if different:")
        if reference_seq != patient_seq:
            print(f"  Reference: {reference_seq[:100]}...")
            print(f"  Patient  : {patient_seq[:100]}...")
        return

    if alignments:
        # Tampilkan alignment terbaik (skor tertinggi)
        # format_alignment menghasilkan string, kita bisa loop untuk tampilan blok
        alignment_str = format_alignment(*alignments[0])
        print(alignment_str.split('\n')[0]) # Skor
        
        aligned_ref = alignments[0][0]
        aligned_pat = alignments[0][1]
        
        block_size = 80
        print(f"\nAligned sequences (Ref vs Patient) for {transcript_id}:")
        for i in range(0, len(aligned_ref), block_size):
            ref_chunk = aligned_ref[i:i+block_size]
            pat_chunk = aligned_pat[i:i+block_size]
            
            match_line = []
            for r, p in zip(ref_chunk, pat_chunk):
                if r == p:
                    match_line.append('|')
                elif r == '-' or p == '-':
                    match_line.append(' ') # Gap
                else:
                    match_line.append('.') # Mismatch
            
            print(f"{ref_chunk} ({i+1}-{min(i+block_size, len(aligned_ref))}) Ref")
            print(f"{''.join(match_line)}")
            print(f"{pat_chunk} ({i+1}-{min(i+block_size, len(aligned_pat))}) Pat")
            print("-" * len(ref_chunk))
    else:
        print("  No alignment generated by pairwise2.")
def main():
    data_dir = "data"
    os.makedirs(data_dir, exist_ok=True)
    fasta_path = os.path.join(data_dir, "DNA_Sequences.fasta")

    if not os.path.exists(fasta_path):
        if FASTA_FILE_GDRIVE_ID:
            print(f"Downloading {fasta_path}...")
            try:
                if os.path.exists(fasta_path): os.remove(fasta_path)
                gdown.download(f"https://drive.google.com/uc?id={FASTA_FILE_GDRIVE_ID}", fasta_path, quiet=False)
                if not os.path.exists(fasta_path): print(f"Download failed for {fasta_path}."); return
            except Exception as e: print(f"Failed to download FASTA: {e}."); return
        else: print(f"FASTA file '{fasta_path}' not found and ID not set."); return

    print("Loading fasta sequences...")
    reference_sequences, transcript_positions = load_fasta_sequences_with_pos(fasta_path)
    if not reference_sequences: print("No sequences loaded from FASTA. Exiting."); return
    print(f"Loaded {len(reference_sequences)} reference sequences.")

    print(f"Loading mutation data for {len(FOCUSED_GENES)} focused genes...")
    all_mutations_df = load_all_mutation_data(data_dir, FOCUSED_GENES, GENE_TSV_GDRIVE_IDS)
    if all_mutations_df.empty: print("No mutation data loaded. Exiting."); return
    print(f"Loaded {len(all_mutations_df)} total mutation records.")

    cds_col_for_parsing = None
    possible_cds_cols = ['MUTATION_CDS', 'CDS_MUTATION_SYNTAX', 'MUTATION_CDS_SYNTAX']
    for col in possible_cds_cols:
        if col in all_mutations_df.columns:
            cds_col_for_parsing = col
            break
    if not cds_col_for_parsing:
        print(f"Error: None of the expected CDS columns ({possible_cds_cols}) found."); return
    
    gene_col = 'GENE_SYMBOL' if 'GENE_SYMBOL' in all_mutations_df.columns else 'GENE_NAME'
    if gene_col not in all_mutations_df.columns:
        print(f"Error: Gene column ('{gene_col}') not found."); return

    mutations_df_focused = all_mutations_df[all_mutations_df[gene_col].isin(FOCUSED_GENES)].copy()
    mutations_df_focused.dropna(subset=['TRANSCRIPT_ACCESSION', 'GENOME_START', cds_col_for_parsing], inplace=True)
    print(f"Processing {len(mutations_df_focused)} mutation records for focused genes after NA drop.")

    processed_count = 0
    for transcript_id, ref_seq_str in reference_sequences.items():
        muts_for_this_transcript = mutations_df_focused[mutations_df_focused['TRANSCRIPT_ACCESSION'] == transcript_id]
        if muts_for_this_transcript.empty: continue
        if transcript_id not in transcript_positions:
            # print(f"Warning: Position info for transcript {transcript_id} not found. Skipping.")
            continue
            
        patient_sequence, applied_mut_info = reconstruct_patient_sequence_with_mutations(
            ref_seq_str, muts_for_this_transcript, 
            transcript_positions[transcript_id], cds_col_for_parsing
        )
        compare_and_display_sequences(patient_sequence, ref_seq_str, transcript_id, applied_mut_info)
        processed_count += 1
            
    print(f"\nFinished processing. Compared {processed_count} transcripts with associated mutation data.")

In [5]:
if __name__ == "__main__":
    main()

Loading fasta sequences...
Loaded 56474 reference sequences.
Loading mutation data for 20 focused genes...
Total records from all TSV files: 70064
Loaded 70064 total mutation records.
Processing 70062 mutation records for focused genes after NA drop.

--- Comparison for Transcript: ENST00000405271.5 ---
Reference Length: 1029, Patient Length: 1029
Applied Mutations (attempted reconstruction):
  - SNV @CDS:'-193' (Genomic:47345179, ListIdx:21): Ref_in_list 'C'(CDS_ref:'C') -> Alt 'T'
  - SNV @CDS:'-158' (Genomic:47345214, ListIdx:56): Ref_in_list 'G'(CDS_ref:'C') -> Alt 'T'
  - SNV @CDS:'-149+240' (Genomic:47345463, ListIdx:305): Ref_in_list 'T'(CDS_ref:'C') -> Alt 'T'
  - SNV @CDS:'-149+725' (Genomic:47345948, ListIdx:790): Ref_in_list 'A'(CDS_ref:'T') -> Alt 'C'
  - SNV @CDS:'-149+745' (Genomic:47345968, ListIdx:810): Ref_in_list 'G'(CDS_ref:'C') -> Alt 'A'
  - SNV @CDS:'-149+747' (Genomic:47345970, ListIdx:812): Ref_in_list 'T'(CDS_ref:'T') -> Alt 'C'
  - SNV @CDS:'-149+765' (Genomic